blink - movs: 65 66 69 - 40ms

Plots: 
- time response: pitch_dot_dot ~ 30ms
                 zero_x: ~ 85 ms
                 min_CM_dot_dot: ~ 75 ms
- linear correletion: pitch vs CM_dot_dot
- correction of velocity direction: time series: CM_x_dot_projected
                                                 delta angle between initial CM_x_dot_projected direction and final
                                    histogram :  initial angle (mean on frames) - final angle(mean on frames)
                        - memory? 
                        - correction time from light? 
                        - correct in dark? in 100 ms? not sure

- they try to keep delta yaw 0 and go back to the same velocity. its not the same target! 
they have velocity in the Y direction (slip) when they react to dark, they slip for a while and continue the according to their position 
most of them will keep the same yaw (it means they need to mesure their angular position some how and keep it the same)
- they manage to keep the same yaw while changing the velocity 
(kind of balance - they use the wings to slip but keep the yaw the same, something symetric should happen)
- they go back to the same velocity size, is it their baseline or they remember?

- when in total darkness they manage to keep the yaw but the velocity doesnt neceseely stay the same (which means it adds slip). 
* they mesure gravity - for pitch? how they keep the same yaw? (not all of them keep the same yaw, less than the pulse one) 
* velocity mesurment - oprical flow and probably antena. the fact that some of them regain the same velocity means that optical flow is not the only sensor

--- the dark response: pitch up and backwards fly  - they try to keep the same yaw and go back to the same velocity. 
if the light turns on they manage to regain the same velocity. they probably keep the same direction as a delta t from the lighs turning on

In [30]:

from Experiment import Experiment
import matplotlib.pyplot as plt
import Plotters 
import plotly.graph_objects as go
import plotly
import numpy as np

import time
import matplotlib.cm as colormap

%matplotlib qt


loadir = 'H:/My Drive/dark 2022/csv_dark' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name =['manipulated_2022_02_03_dark','manipulated_2023_08_09_60ms']

# ['manipulated_2023_08_07_10ms',
#             'manipulated_2023_08_06_40ms',
#            'manipulated_2023_08_09_60ms',
#            'manipulated_2023_08_09_80ms',
#            'manipulated_2023_08_10_100ms',
#            'manipulated_2022_02_03_dark',
#            'manipulated_2022_01_31_darkdown']




def dont_show_movs(exp_name,movs_to_del):
    return [mov_name for mov_name in experiments[exp_name].mov_names if mov_name not in movs_to_del ] 

mov_dark_delete = ['mov30','mov40','mov57','mov59']
mov_100_delete = ['mov10','mov102','mov19','mov34','mov35','mov47','mov51','mov52','mov66','mov68','mov73','mov90']
mov_80_delete = ['mov119','mov49','mov97','mov99']
mov_40_delete = ['mov64']
mov_60_delete = ['mov12','mov13','mov147','mov165','mov21','mov24','mov26','mov48','mov86']
mov_10_delete = ['mov46','mov54']
mov_5_delete = ['mov44']


color_map = colormap.datad["tab10"]['listed']

experiments = {exp:Experiment(loadir,exp) for exp in exp_name} # load experiments


In [25]:

[exp.xy_body_on_sp_movies()  for exp in experiments.values()] # smooth and derive


[None]

Smooth, calculate projected axes and angles

In [31]:
[exp.from_vector_to_wing_body('Y_x_body',['vectors','body'])  for exp in experiments.values()] # smooth and derive


[exp.smooth_prop_movies('CM_real_x_body',['','_dot','_dot_dot'],'body')  for exp in experiments.values()] # smooth and derive


[exp.project_axes_movies(['X_x_body_projected','X_y_body_projected'], add_to_vectors = ['vectors','body'])  for exp in experiments.values()] # X_body axis projected to xy plane
[exp.project_axes_movies(['Y_x_body_projected','Y_y_body_projected'],axis = 'Y_x_body',add_to_vectors = ['vectors','body'])  for exp in experiments.values()] # X_body axis projected to xy plane

[exp.project_axes_movies(['CM_dot_xax','CM_dot_yax'],axis = 'CM_real_x_body_dot_smth',wing_body = 'body',add_to_vectors = ['body','vectors'])  for exp in experiments.values()] # Velocity direction in the projected X_body axes (xy plane) 

# calculate the size of the velocity projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected',add_to_vectors = True)  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected',add_to_vectors = True)  for exp in experiments.values()]


# calculate the size of the acceleration projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected')  for exp in experiments.values()]

[exp.pqr_movies() for exp in experiments.values()] # calculate p,q,r (angular velocity in body axis)

[exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi') for exp in experiments.values()] # calculate the mean value of two properties


[exp.substruct_first_frame('yaw_body','body')  for exp in experiments.values()] # substract the zero frame from a property ( {prop}_min_ref_frame )

# calculate the difference between current direction and another one at the same time:
# the angle between Xbody on the xy plane and the velocity direction on the xy plane 
[exp.delta_ang_all_time_movies('X_x_body_projected','CM_dot_xax','slip_angle_beta',three_col = 2) for exp in experiments.values()] 

# calculate the difference in angle between current property and in referance frame:
# the angle between Xbody on the xy (in 0 time) plane and Xbody on xy plane 
[exp.delta_ref_ang_movies('X_x_body_projected','delta_ang_Xax',axis = 'X_x_body_projected')  for exp in experiments.values()]

# the angle between Xbody on the xy (in 0 time) plane and the velocity direction on the xy plane 
[exp.delta_ref_ang_movies('CM_dot_xax','delta_ang_CM_dot_x',axis = 'CM_dot_xax',delta_ang = 0) for exp in experiments.values()]
[exp.project_prop_all_axes_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_all_axes_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected')  for exp in experiments.values()]



[exp.project_prop_all_axes_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_all_axes_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]

[exp.project_prop_all_axes_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y_body',three_col = 2,ax_to_proj = 'Y_x_body')  for exp in experiments.values()]

[exp.calculate_phi_amp_movies('rw','mean_body')  for exp in experiments.values()]
[exp.calculate_phi_amp_movies('lw','mean_body')  for exp in experiments.values()]
[exp.calculate_freq_movies('phi_rw_min_idx', 'mean_body')  for exp in experiments.values()]
[exp.calculate_freq_movies('phi_lw_min_idx', 'mean_body')  for exp in experiments.values()]



props_to_mean = ['x','y','z']
[[exp.mean_by_stroke_movies(f'CM_real_{x}_body_dot_dot_smth','mean_body','body')  for exp in experiments.values()] for x in props_to_mean]
[[exp.mean_by_stroke_movies(f'CM_dot_dot_{x}_projected_all_axes','mean_body','body')  for exp in experiments.values()] for x in ['x','y']]

[[exp.mean_by_stroke_movies(f'X_{x}_body_projected','mean_body','body')  for exp in experiments.values()] for x in ['x','y']]
[[exp.mean_by_stroke_movies(f'Y_{x}_body_projected','mean_body','body')  for exp in experiments.values()] for x in ['x','y']]



c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:124: RuntimeWarning:

invalid value encountered in arccos

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:369: RuntimeWarning:

divide by zero encountered in divide



Calculate model and project to projected body axes

In [17]:
[exp.calc_drag_movies() for exp in experiments.values()]

# [exp.calculate_model_movies(add_drag = True,g = 13) for exp in experiments.values()]
[exp.calculate_model_movies(add_drag = False) for exp in experiments.values()]

[None]

In [10]:
# subplot of heli model

exp = experiments['manipulated_2023_08_09_60ms']
from plotly.subplots import make_subplots
fig_subplots = make_subplots(rows=3, cols=1, subplot_titles= ['acc_x','acc_y','acc_z'])
for idx,x in enumerate(['x','y','z']):
    fig = go.Figure()

    prop_name = [f'model_gamma_{x}_no_drag_projected_all_axes',f'model_{x}_no_drag_projected_all_axes',
                 f'CM_dot_dot_{x}_projected_all_axes']
    if x == 'z':
        prop_name = [f'model_gamma_{x}_no_drag',f'model_{x}_no_drag',f'CM_real_{x}_body_dot_dot_smth']

    exp.plot_prop_movies(prop_name[0],'mean_body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,
                         name = 'gamma model no drag - ',showlegend = idx==0,marker_color = 'rgb(255, 100, 100)',
                         marker_size = 3,mode = 'markers+lines')
    exp.plot_prop_movies(prop_name[1],'body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,
                         showlegend = idx==0,name = 'gamma model drag - ',marker_size = 7,mode = 'markers+lines',
                         symbol = 'cross',marker_color = 'rgb(100, 255, 100)')
    exp.plot_prop_movies(prop_name[2],'body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,name = 'experiment - ',
                         showlegend = idx==0,mode = 'markers+lines',symbol = 'triangle-up',marker_color = 'rgb(0, 0, 0)',marker_size = 3)
    traces = fig.data

    # Add traces to the subplots
    for trace in traces:
        fig_subplots.add_trace(trace, row=idx + 1, col=1)

    # Update layout of subplots
fig_subplots.update_layout(title='Subplots with Extracted Traces')
plotly.offline.plot(fig_subplots, filename=f'{exp.figures_path}/with_drag_plot.html',auto_open=False)

# Display the subplots
fig_subplots.show()

In [11]:
# subplot of heli model

exp = experiments['manipulated_2023_08_09_60ms']
from plotly.subplots import make_subplots
fig_subplots = make_subplots(rows=3, cols=1, subplot_titles= ['acc_x','acc_y','acc_z'])
for idx,x in enumerate(['x','y','z']):
    fig = go.Figure()

    prop_name = [f'model_gamma_{x}_no_drag',f'model_gamma_{x}_drag',
                 f'CM_real_{x}_body_dot_dot_smth']
    if x == 'z':
        prop_name = [f'model_gamma_{x}_no_drag',f'model_gamma_{x}_drag',f'CM_real_{x}_body_dot_dot_smth']

    exp.plot_prop_movies(prop_name[0],'mean_body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,
                         name = 'gamma model no drag - ',showlegend = idx==0,marker_color = 'rgb(255, 100, 100)',
                         marker_size = 3,mode = 'markers+lines')
    exp.plot_prop_movies(prop_name[1],'mean_body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,
                         showlegend = idx==0,name = 'gamma model drag - ',marker_size = 7,mode = 'markers+lines',
                         symbol = 'cross',marker_color = 'rgb(100, 255, 100)')
    exp.plot_prop_movies(prop_name[2],'body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,name = 'experiment - ',
                         showlegend = idx==0,mode = 'markers+lines',symbol = 'triangle-up',marker_color = 'rgb(0, 0, 0)',marker_size = 3)
    traces = fig.data

    # Add traces to the subplots
    for trace in traces:
        fig_subplots.add_trace(trace, row=idx + 1, col=1)

    # Update layout of subplots
fig_subplots.update_layout(title='Subplots with Extracted Traces')
plotly.offline.plot(fig_subplots, filename=f'{exp.figures_path}/with_drag_plot.html',auto_open=False)

# Display the subplots
fig_subplots.show()

KeyError: 'model_gamma_x_drag'

Subplots of heli model

error and RMS - Model vs exp

In [23]:
# error and RMS - Model vs exp


drag = 'no_drag'
exp = experiments['manipulated_2023_08_09_60ms']
# z_model = exp.diff_model_exp_movies(f'model_z_{drag}','CM_real_z_body_dot_dot_smth')
# z_model_correct = exp.diff_model_exp_movies(f'model_gamma_z_{drag}','CM_real_z_body_dot_dot_smth')


x_model_correct = exp.diff_model_exp_movies(f'model_gamma_x_no_drag_projected_all_axes','CM_dot_dot_x_projected_all_axes')

y_model_correct = exp.diff_model_exp_movies(f'model_gamma_y_no_drag_projected_all_axes','CM_dot_dot_y_projected_all_axes')


fig,ax = plt.subplots(1,3)
range_bins = [-6,6]
ax[0].hist(x_model_correct[0][:,1],bins = 100,alpha = 0.5,range = range_bins)

ax[1].hist(y_model_correct[0][:,1],bins = 100,alpha = 0.5,range = range_bins)

ax[2].hist(z_model_correct[0][:,1],bins = 100,alpha = 0.5,range = range_bins)
ax[2].hist(z_model[0][:,1],bins = 50,alpha = 0.5,range = range_bins)

plt.legend(['model','corrected'])


In [29]:
# error and RMS - Model vs exp

drag = 'no_drag'
exp = experiments['manipulated_2023_08_09_60ms']
z_model = exp.diff_model_exp_movies(f'model_z_{drag}','CM_real_z_body_dot_dot_smth')
z_model_correct = exp.diff_model_exp_movies(f'model_gamma_z_{drag}','CM_real_z_body_dot_dot_smth')


x_model = exp.diff_model_exp_movies(f'model_x_{drag}','CM_real_x_body_dot_dot_smth')
x_model_correct = exp.diff_model_exp_movies(f'model_gamma_x_{drag}','CM_real_x_body_dot_dot_smth')

y_model = exp.diff_model_exp_movies(f'model_y_{drag}','CM_real_y_body_dot_dot_smth')
y_model_correct = exp.diff_model_exp_movies(f'model_gamma_y_{drag}','CM_real_y_body_dot_dot_smth')


fig,ax = plt.subplots(1,3)
range_bins = [-6,6]
ax[0].hist(x_model_correct[0][:,1],bins = 50,alpha = 0.5,range = range_bins)
ax[0].hist(x_model[0][:,1],bins = 50,alpha = 0.5,range = range_bins)

ax[1].hist(y_model_correct[0][:,1],bins = 50,alpha = 0.5,range = range_bins)
ax[1].hist(y_model[0][:,1],bins = 50,alpha = 0.5,range = range_bins)

ax[2].hist(z_model_correct[0][:,1],bins = 50,alpha = 0.5,range = range_bins)
ax[2].hist(z_model[0][:,1],bins = 50,alpha = 0.5,range = range_bins)

plt.legend(['model','corrected'])


In [11]:
mov = exp.get_mov('mov101')
mov.header['body']

{'pitch_body': 0,
 'yaw_body': 1,
 'roll_body': 2,
 'pitch_body_dot': 3,
 'yaw_body_dot': 4,
 'roll_body_dot': 5,
 'pitch_body_dot_dot': 6,
 'yaw_body_dot_dot': 7,
 'roll_body_dot_dot': 8,
 'frames': 9,
 'time': 10,
 'CM_real_x_body': 11,
 'CM_real_y_body': 12,
 'CM_real_z_body': 13,
 'phi_rw_max_idx': 14,
 'phi_rw_max_val': 15,
 'phi_lw_max_idx': 16,
 'phi_lw_max_val': 17,
 'phi_rw_min_idx': 18,
 'phi_rw_min_val': 19,
 'phi_lw_min_idx': 20,
 'phi_lw_min_val': 21,
 'yaw_z_frame': 22,
 'pitch_y_frame': 23,
 'roll_x_frame': 24,
 'Y_x_body': 25,
 'CM_real_x_body_smth': 26,
 'CM_real_y_body_smth': 27,
 'CM_real_z_body_smth': 28,
 'CM_real_x_body_dot_smth': 29,
 'CM_real_y_body_dot_smth': 30,
 'CM_real_z_body_dot_smth': 31,
 'CM_real_x_body_dot_dot_smth': 32,
 'CM_real_y_body_dot_dot_smth': 33,
 'CM_real_z_body_dot_dot_smth': 34,
 'X_x_body_projected': 35,
 'X_y_body_projected': 36,
 'Y_x_body_projected': 37,
 'Y_y_body_projected': 38,
 'CM_dot_xax': 39,
 'CM_dot_yax': 40,
 'CM_dot_x_projec

In [ ]:
fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('pitch_body','body',fig,case = 'plot_mov',prop_x = 'CM_dot_dot_x_projected_all_axes' ,add_horizontal_line=None,name = 'calculated')
fig.show()

fig = go.Figure()
exp = experiments['manipulated_2023_08_09_60ms']
exp.plot_prop_movies('pitch_body','body',fig,case = 'plot_mov',prop_x = 'CM_dot_dot_x_projected_all_axes' ,add_horizontal_line=None,name = 'calculated')
fig.show()

In [26]:
diff_mod_exp = []
exp = experiments['manipulated_2023_08_09_60ms']
wing_body = 'mean_body'
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    model_x = (mov.get_prop('model_gamma_x_drag',wing_body))
    exp_x = (mov.get_prop('CM_real_x_body_dot_dot_smth',wing_body))
    diff_mod_exp.append(np.arcsin((model_x-exp_x)/exp_x/9.8))
plt.hist(np.vstack(diff_mod_exp),bins = 50)
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    model_x = (mov.get_prop('model_gamma_z_drag',wing_body))
    exp_x = (mov.get_prop('CM_real_z_body_dot_dot_smth',wing_body))
    diff_mod_exp.append(np.arcsin((model_x-exp_x)/exp_x/9.8))
plt.hist(np.vstack(diff_mod_exp),bins = 50,alpha = 0.5)


C:\Users\Roni\AppData\Local\Temp\ipykernel_20612\1238036132.py:8: RuntimeWarning:

invalid value encountered in arcsin

C:\Users\Roni\AppData\Local\Temp\ipykernel_20612\1238036132.py:14: RuntimeWarning:

invalid value encountered in arcsin



(array([2.000e+00, 1.000e+00, 4.000e+00, 1.000e+01, 1.100e+01, 9.000e+00,
        1.400e+01, 1.600e+01, 9.000e+00, 1.800e+01, 2.400e+01, 3.000e+01,
        3.800e+01, 5.300e+01, 6.400e+01, 6.500e+01, 9.400e+01, 1.150e+02,
        1.320e+02, 1.580e+02, 2.140e+02, 3.670e+02, 6.190e+02, 1.416e+03,
        2.127e+03, 1.656e+03, 7.720e+02, 3.900e+02, 2.550e+02, 1.960e+02,
        1.580e+02, 1.230e+02, 7.800e+01, 7.400e+01, 6.100e+01, 5.500e+01,
        3.900e+01, 2.800e+01, 2.900e+01, 2.400e+01, 1.000e+01, 1.500e+01,
        1.800e+01, 1.900e+01, 1.400e+01, 8.000e+00, 6.000e+00, 6.000e+00,
        2.000e+00, 1.000e+00]),
 array([-1.55622117, -1.49425666, -1.43229215, -1.37032764, -1.30836313,
        -1.24639862, -1.18443411, -1.12246959, -1.06050508, -0.99854057,
        -0.93657606, -0.87461155, -0.81264704, -0.75068253, -0.68871802,
        -0.62675351, -0.56478899, -0.50282448, -0.44085997, -0.37889546,
        -0.31693095, -0.25496644, -0.19300193, -0.13103742, -0.0690729 ,
        -0.

In [33]:
exp = experiments['manipulated_2023_08_09_60ms']
fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('acc_zlab_g','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,name = 'calculated')
exp.plot_prop_movies('CM_real_z_body_dot_dot_smth','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,mode = 'markers+lines',name = 'measured',marker_size = 6)

fig.show()

In [34]:
[exp.calc_force_movies()  for exp in experiments.values()]


[None, None, None, None, None, None, None]

In [35]:
exp = experiments['manipulated_2023_08_09_60ms']
fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','body',fig,case = 'plot_mov',prop_x = 'CM_real_z_body_dot_dot_smth' ,add_horizontal_line=None)

fig.show()

In [36]:
exp = experiments['manipulated_2022_01_31_darkdown']
fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('force_size','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)

fig.show()

fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('CM_dot_dot_x_projected_all_axes','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)

fig.show()

fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)

fig.show()

In [37]:
exp = experiments['manipulated_2022_02_03_dark']

fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('CM_dot_dot_x_projected_all_axes','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)

fig.show()

In [38]:
[exp.substruct_first_frame('CM_real_x_body_smth','body')  for exp in experiments.values()] # substract the zero frame from a property ( {prop}_min_ref_frame )
[exp.substruct_first_frame('CM_real_y_body_smth','body')  for exp in experiments.values()] # substract the zero frame from a property ( {prop}_min_ref_frame )


[None, None, None, None, None, None, None]

In [39]:

exp = experiments['manipulated_2022_02_03_dark']

fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('CM_real_x_body_smth_min_ref_frame','body',fig,case = 'plot_mov',prop_x = 'CM_real_y_body_smth_min_ref_frame' ,add_horizontal_line=None)

fig.show()

In [40]:
exp = experiments['manipulated_2023_08_09_60ms']
mov = exp.get_mov('mov101')



ay = mov.get_prop('CM_real_y_body_dot_dot_smth','body')
ax = mov.get_prop('CM_real_x_body_dot_dot_smth','body')
az = mov.get_prop('CM_real_z_body_dot_dot_smth','body')



force = np.hstack((ax,ay,az))
force_norm = (force.T/np.linalg.norm(force,axis = 1)).T
# force_size = dict(force_size = np.sqrt(np.sum(force**2,axis = 1)))

exp = experiments['manipulated_2022_02_03_dark']
mov.plot_3d_traj_movie('CM_real_y_body_dot_dot_smth',forces = force_norm)



In [13]:
exp = experiments['manipulated_2023_08_09_60ms']
fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('roll_body','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('delta_ang_Xax','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)

fig.show()

phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)

fig = go.Figure()
exp.plot_prop_movies('theta_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('theta_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)

exp.plot_prop_movies('psi_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('psi_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)


exp.plot_prop_movies('phi_lw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_l_color )
exp.plot_prop_movies('phi_rw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_r_color)

fig.show()



In [42]:

fig = go.Figure()
exp.plot_prop_movies('CM_real_y_body_dot_dot_smth','body',fig,case = 'plot_mov',prop_x = 'time' )

fig.show()


In [43]:
idx_prop = 'phi_rw_min_idx'
mean_wing_body = 'mean_body'
[exp.calculate_freq_movies(idx_prop, 'mean_body')  for exp in experiments.values()]

idx_prop = 'phi_lw_min_idx'
[exp.calculate_freq_movies(idx_prop, 'mean_body')  for exp in experiments.values()]

prop = 'CM_dot_dot_x_projected'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]


prop = 'CM_dot_x_projected'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]
prop = 'delta_ang_CM_dot_x'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]
prop = 'delta_ang_Xax'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]


prop = 'CM_dot_y_projected_all_axes'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]


[exp.sub_two_props_movies('delta_ang_Xax','roll_body','body',['delta_roll_yaw'])  for exp in experiments.values()]





c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:379: RuntimeWarning:

divide by zero encountered in divide



[None, None, None, None, None, None, None]

In [44]:

prop = 'CM_dot_y_projected_all_axes'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]


[None, None, None, None, None, None, None]

In [45]:
prop = 'CM_dot_dot_x_projected_all_axes'
[exp.mean_by_stroke_movies(prop,'mean_body','body')  for exp in experiments.values()]


[None, None, None, None, None, None, None]

In [46]:
[exp.calculate_phi_amp_movies('rw',mean_wing_body)  for exp in experiments.values()]
[exp.calculate_phi_amp_movies('lw',mean_wing_body)  for exp in experiments.values()]


[None, None, None, None, None, None, None]

In [47]:
[exp.sub_two_props_movies('phi_lw_max_val','phi_rw_max_val','mean_body',['delta_max_phi'])  for exp in experiments.values()]
[exp.sub_two_props_movies('phi_lw_min_val','phi_rw_min_val','mean_body',['delta_min_phi'])  for exp in experiments.values()]
[exp.sub_two_props_movies('amp_lw','amp_rw','mean_body',['delta_amp'])  for exp in experiments.values()]


[exp.sub_two_props_movies('freq_phi_lw_min_idx','freq_phi_rw_min_idx','mean_body',['delta_freq'])  for exp in experiments.values()]



[None, None, None, None, None, None, None]

In [48]:
fig = go.Figure()
exp.plot_prop_movies('delta_ang_Xax','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,mode = 'markers')
fig.show()



In [49]:
fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,mode = 'markers')
fig.show()


In [50]:
fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,mode = 'markers')
exp.plot_prop_movies('roll_body','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,mode = 'markers')
fig.show()

In [51]:

fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','body',fig,case = 'plot_mov',prop_x = 'roll_body' ,add_horizontal_line=None,mode = 'markers')
fig.show()


fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_x_projected_all_axes','body',fig,case = 'plot_mov',prop_x = 'pitch_body' ,add_horizontal_line=None,mode = 'markers')
fig.show()



In [52]:
exp = experiments['manipulated_2023_08_09_60ms']

fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','mean_body',fig,case = 'plot_mov',prop_x = 'p' ,add_horizontal_line=None,mode = 'markers')
fig.show()


fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','mean_body',fig,case = 'plot_mov',prop_x = 'delta_max_phi' ,add_horizontal_line=None,mode = 'markers')
fig.show()


fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','mean_body',fig,case = 'plot_mov',prop_x = 'delta_amp' ,add_horizontal_line=None,mode = 'markers')
fig.show()



fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','mean_body',fig,case = 'plot_mov',prop_x = 'delta_freq' ,add_horizontal_line=None,mode = 'markers')
fig.show()

KeyError: 'p'

In [ ]:

fig = go.Figure()
exp.plot_prop_movies('delta_ang_Xax','mean_body',fig,case = 'plot_mov',prop_x = 'delta_amp' ,add_horizontal_line=None,mode = 'markers')
fig.show()

In [ ]:
fig = go.Figure()

exp.plot_prop_movies('roll_body','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

In [ ]:

fig = go.Figure()
exp.plot_prop_movies('delta_amp','mean_body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,mode = 'markers')

exp.plot_prop_movies('delta_ang_Xax','mean_body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']

fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('CM_dot_dot_x_projected','body',fig,case = 'plot_mov',prop_x = 'pitch_body' ,add_horizontal_line=None,mode = 'markers')

fig.show()

In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']

fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('delta_roll_yaw','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('delta_ang_Xax','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)

fig.show()

In [ ]:
from plotly.subplots import make_subplots
time_vec = [10,60,100,220,280]

xbins = dict(start=0,end=360,size = 10)
fig = Plotters.subplot_histograms_delta_prop(time_vec,experiments,'delta_ang_CM_dot_x',color_map,xbins,'body')
fig.show()





Violin plot

In [ ]:
exp.figures_path

'H:/My Drive/dark 2022/csv_dark/figures/2023_08_09_60ms'

In [ ]:


t_vec = [0,20,75,150,200]
delta_t = 73*5/16000*1000
prop = 'freq_phi_rw_min_idx'
data = [exp.mean_prop_time_vector_movies(prop,delta_t,t_vec, mov = False) for exp in  experiments.values()]


fig = go.Figure()
[Plotters.vplot(dat,fig,'t0','freq_phi_rw_min_idx',exp_name) for dat,exp_name in zip(data,experiments.keys())]
fig.show()


prop = 'CM_dot_x_projected'
data = [exp.mean_prop_time_vector_movies(prop,delta_t,t_vec, mov = False,sub_mean = False) for exp in  experiments.values()]


fig = go.Figure()
[Plotters.vplot(dat,fig,'t0','CM_dot_x_projected',exp_name) for dat,exp_name in zip(data,experiments.keys())]
fig.show()

plotly.offline.plot(fig, filename=f'I:/My Drive/Research/Dark/article/figures/html/CM_dot_x_violin_plot.html',auto_open=False)



c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:343: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:343: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:343: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:343: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:343: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:343: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:343: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:343: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:343: RuntimeWarning:

Mean of empty slice

c:\Users\R

'I:/My Drive/Research/Dark/article/figures/html/CM_dot_x_violin_plot.html'

In [ ]:
prop = 'CM_real_z_body_dot_dot_smth'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]



[None, None, None, None, None, None]

Histograms

In [ ]:
exp = experiments[list(experiments.keys())[0]]
[exp.get_peaks_movies('q','max') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','peaks_max',t0 = 40, t1 = 100) for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame','max') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','zero_v') for exp in experiments.values()]
[exp.get_peaks_movies('delta_ang_Xax','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','peaks_max',t0 = 5, t1 = 100) for exp in experiments.values()]

[exp.get_peaks_movies('CM_dot_x_projected','half_v') for exp in experiments.values()]

prop  = 'CM_dot_x_projected_zero_v'
[exp.subtract_interest_time_from_time(prop,'time_m_zero_v') for exp in experiments.values()]

point_name = 'CM_dot_x_projected_zero_v'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])

fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_x_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_x_projected_peaks_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


In [ ]:
exp = experiments['manipulated_2023_08_06_40ms']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']
# exp_name =['manipulated_2023_08_07_10ms',
#             'manipulated_2023_08_06_40ms',
#            'manipulated_2023_08_09_60ms',
#            'manipulated_2023_08_09_80ms',
#            'manipulated_2023_08_10_100ms',
#            'manipulated_2022_02_03_dark']

prop = 'delta_ang_CM_dot_x'
fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time_m_zero_v')
fig.show()


prop = 'delta_ang_Xax'
fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

prop = 'delta_ang_CM_dot_x'
fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
exp.plot_prop_movies('delta_ang_Xax','body',fig,case = 'plot_mov',prop_x = 'time')

fig.show()


fig.show()
fig = go.Figure()
exp.plot_prop_movies('freq_phi_rw_min_idx','mean_body',fig,case = 'plot_mov',prop_x = 'time' ,
                     add_horizontal_line=None)
fig.show()



KeyError: 'time_m_zero_v'

In [ ]:
from scipy.signal import argrelextrema, savgol_filter,find_peaks

def dot_interp(data):
    # sgn = np.sign(np.cross(vector,data))[:,-1]

    
    not_nan = np.logical_not(np.isnan(data))
    indices = np.arange(len(data))
    return np.interp(indices, indices[not_nan], data[not_nan])[::,np.newaxis]


exp = experiments['manipulated_2023_08_09_60ms']
mov = exp.get_mov('mov101')
time = mov.get_prop(f'time','vectors')
cm = mov.get_prop(f'CM_real_x_body','body',three_col=3)

deriv = 1

fig,ax = plt.subplots(4,1,sharex=True)

for wing in ['lw','rw']:
    tip_rw = mov.get_prop(f'tip_x_{wing}','vectors',three_col=3) 
    tip_wing_ax = np.hstack([dot_interp(tip_rw[:,idx]) for idx in range(tip_rw.shape[1])])

    deriv = 1
    smoothed_tip_dot = savgol_filter(tip_wing_ax.T/mov.dt**deriv,71,2,deriv = deriv).T



    chord_rw = mov.get_prop(f'chord_x_{wing}','vectors',three_col=3)
    span_rw = mov.get_prop(f'span_x_{wing}','vectors',three_col=3)
    z_wing = np.cross(span_rw,chord_rw)
    z_wing = z_wing/np.linalg.norm(z_wing,axis = 1)[::,np.newaxis]

    deriv = 0
    # chord_rw = savgol_filter(chord_rw.T/mov.dt**deriv,3,2,deriv = deriv).T
    # span_rw = savgol_filter(span_rw.T/mov.dt**deriv,3,2,deriv = deriv).T
    # z_wing = savgol_filter(z_wing.T/mov.dt**deriv,3,2,deriv = deriv).T

    projected = np.vstack([np.sum(vector*smoothed_tip_dot,axis = 1) for vector in [span_rw,chord_rw,z_wing]]).T

    # angles = np.arctan(projected[:,2]/projected[:,1])
    # shifted_angles = np.where(angles < 0, angles +np.pi, angles)

    
    angles = np.arctan2(projected[:,2],projected[:,1])
    # shifted_angles = np.where(angles < 0, angles +np.pi, angles)
    shifted_angles = np.abs(angles)
    # shifted_angles_body = np.unwrap(angles_body,discont = np.pi,period = 6*np.pi)
    psi = mov.get_prop(f'psi_{wing}','wing')
    phi = mov.get_prop(f'phi_{wing}','wing')
    theta = mov.get_prop(f'theta_{wing}','wing')

    # shifted_angles_body = angles_body
    if wing == 'lw':

        # ax[0].plot(time,-np.arctan(smoothed[2],smoothed[1]),'-*',color = 'blue')

        ax[0].plot(time,shifted_angles*180/np.pi,'-*',color = 'blue')
        ax[1].plot(time,psi,'-*',color = 'blue')
        ax[2].plot(time,phi,'-*',color = 'blue')
        ax[3].plot(time,theta,'-*',color = 'blue')

    else:
        ax[0].plot(time,shifted_angles*180/np.pi,'-*',color = 'red')

        ax[1].plot(time,psi,'-*',color = 'red')

        ax[2].plot(time,phi,'-*',color = 'red')

        ax[3].plot(time,theta,'-*',color = 'red')

ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
# # plt.figure()
#     plt.plot(time,smoothed_cm.T[:,0]/10)


ax = plt.figure().add_subplot(111,projection='3d')
ax.set_aspect('equal')
color = ['r','b','k','g']
label_vec = ['s','c','z','u']
boxsize = 0.05
tm = [0,10,20]
for vector,clr,lbl in zip([span_rw,chord_rw,z_wing,smoothed_tip_dot],color,label_vec):
    for t in tm:

        vec = vector[t]/np.linalg.norm(vector[t])
        u,v,w = vec/100

        x,y,z = cm[t]*10

        ax.plot([x,x+u], [y,y+v], [z,z+w],color = clr,label = lbl)
    ax.set_zlim(-boxsize, boxsize)

    ax.set_ylim(-boxsize, boxsize)
    ax.set_xlim(-boxsize, boxsize)
ax.legend()

In [ ]:
from scipy.signal import argrelextrema, savgol_filter,find_peaks
exp = experiments['manipulated_2023_08_09_60ms']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']
t0 = 12
t1 = 70
fig,ax = plt.subplots(2,1,sharex = True)
fig2,ax2 = plt.subplots(1,8)
fig1,ax1 = plt.subplots(1,1)

ini_time = []
slip_time = []
delta_t = 300
# mov29, mov21
for mov_name in dict_v['lt_zero']:
    mov = exp.get_mov(mov_name)
    delta_ang_CM_dot_x = mov.get_prop('delta_ang_CM_dot_x','body')[:,0]
    pitch = mov.get_prop('pitch_body','body')[:,0]

    delta_ang_CM_dot_x[np.isnan(delta_ang_CM_dot_x)] = 0
    time = mov.get_prop('time','body')[:,0]
    delta_roll_yaw = mov.get_prop('delta_roll_yaw','body')[:,0]


    smoothed = savgol_filter(delta_ang_CM_dot_x.T/mov.dt**1,500,3,deriv = 1)

    max_peak = find_peaks(np.abs(smoothed),prominence = 3000)[0]

    if len(max_peak) > 1 :
        ax[1].plot(time,delta_ang_CM_dot_x)
        ax[0].plot(time,delta_roll_yaw)

        ax[1].plot(time[max_peak[0] - delta_t:max_peak[0]+delta_t ],delta_ang_CM_dot_x[max_peak[0] - delta_t:max_peak[0]+delta_t ],color = 'orange')
        ax[1].plot(time[mov.ref_frame - 100:mov.ref_frame+300 ],delta_ang_CM_dot_x[mov.ref_frame - 100:mov.ref_frame+300 ],color = 'orange')
        ax[0].plot(time[max_peak[0] - delta_t:max_peak[0]+delta_t ],delta_roll_yaw[max_peak[0] - delta_t:max_peak[0]+delta_t ],color = 'red')
        ax[0].plot(time[mov.ref_frame - 100:mov.ref_frame+300 ],delta_roll_yaw[mov.ref_frame - 100:mov.ref_frame+300 ],color = 'red')


    if len(max_peak) > 1 :
        
        idx = mov.mean_time(time[[max_peak[0] - delta_t,max_peak[0] + delta_t]],'mean_body', time = 'time')

        idx_ini = mov.mean_time(time[[mov.ref_frame - 100,mov.ref_frame + 300]],'mean_body', time = 'time')

        


        min_val_lw_min = mov.get_prop('phi_lw_min_val','mean_body')[:,0]
        min_val_lw_max = mov.get_prop('phi_lw_max_val','mean_body')[:,0]
        min_val_rw_min = mov.get_prop('phi_rw_min_val','mean_body')[:,0]
        min_val_rw_max = mov.get_prop('phi_rw_max_val','mean_body')[:,0]
        pitch_dot = mov.get_prop('pitch_body_dot_dot','mean_body')[:,0]
        freq_r = mov.get_prop('freq_phi_rw_min_idx','mean_body')[:,0]
        freq_l = mov.get_prop('freq_phi_lw_min_idx','mean_body')[:,0]

        delta_cm_dot = mov.get_prop('delta_ang_CM_dot_x','mean_body')[:,0]
        

        time_mean = mov.get_prop('time','mean_body')[:,0]

        ini_time.append([np.nanmean(min_val_lw_max[idx_ini[0]:idx_ini[1]]),np.nanmean(min_val_lw_min[idx_ini[0]:idx_ini[1]]),
                         np.nanmean(min_val_rw_max[idx_ini[0]:idx_ini[1]]),np.nanmean(min_val_rw_min[idx_ini[0]:idx_ini[1]]),
                         np.nanmean(freq_l[idx_ini[0]:idx_ini[1]]),np.nanmean(freq_r[idx_ini[0]:idx_ini[1]]),
                         np.nanmean(pitch_dot[idx_ini[0]:idx_ini[1]]),np.nanmean(delta_cm_dot[idx_ini[0]:idx_ini[1]])]) 
        
        
        slip_time.append([np.nanmean(min_val_lw_max[idx[0]:idx[1]]),np.nanmean(min_val_lw_min[idx[0]:idx[1]]),
                          np.nanmean(min_val_rw_max[idx[0]:idx[1]]),np.nanmean(min_val_rw_min[idx[0]:idx[1]]),
                          np.nanmean(freq_l[idx[0]:idx[1]]),np.nanmean(freq_r[idx[0]:idx[1]]),
                          np.nanmean(pitch_dot[idx[0]:idx[1]]),np.nanmean(delta_cm_dot[idx[0]:idx[1]])]) 
        
ini_time = np.array(ini_time)
slip_time = np.array(slip_time)
idx4plot = range(slip_time.shape[0])#np.where(np.abs(slip_time[:, 7]) > 100)[0]
bins = 10


ax2[0].hist((ini_time[idx4plot,0] + ini_time[idx4plot,2])/2,alpha = 0.5,bins = bins,stacked = True)
ax2[0].hist((slip_time[idx4plot,0] + slip_time[idx4plot,2])/2,alpha = 0.5,bins = bins,stacked = True)
plt.legend(['ini','slip'])
ax2[0].set_title('mean_max')


ax2[1].hist((ini_time[idx4plot,1] + ini_time[idx4plot,3])/2,alpha = 0.5,bins = bins,stacked = True)
ax2[1].hist((slip_time[idx4plot,1] + slip_time[idx4plot,3])/2,alpha = 0.5,bins = bins,stacked = True)
plt.legend(['ini','slip'])
ax2[1].set_title('mean_min')


ax2[2].hist(((ini_time[idx4plot,0] - ini_time[idx4plot,1]) + (ini_time[idx4plot,2] - ini_time[idx4plot,3]))/2,
            alpha = 0.5,bins = bins,stacked = True)
ax2[2].hist(((slip_time[idx4plot,0] - slip_time[idx4plot,1]) + (slip_time[idx4plot,2] - slip_time[idx4plot,3]))/2,
            alpha = 0.5,bins = bins,stacked = True)
plt.legend(['ini','slip'])
ax2[2].set_title('mean_amp')


ax2[3].hist(ini_time[idx4plot,6],alpha = 0.5,bins = bins,stacked = True)
ax2[3].hist(slip_time[idx4plot,6],alpha = 0.5,bins = bins,stacked = True)
plt.legend(['ini','slip'])
ax2[3].set_title('pitch_dot_dot')



ax2[4].hist(np.abs(ini_time[idx4plot,5] - ini_time[idx4plot,4]),alpha = 0.5,bins = bins,stacked = True,range = (0,5))
ax2[4].hist(np.abs(slip_time[idx4plot,5] - slip_time[idx4plot,4]),alpha = 0.5,bins = bins,stacked = True,range = (0,5))
plt.legend(['ini','slip'])
ax2[4].set_title('freq_r - freq_l')



ax2[5].hist(ini_time[idx4plot,0] - ini_time[idx4plot,2] ,alpha = 0.5,bins = bins,stacked = True)
ax2[5].hist(slip_time[idx4plot,0] - slip_time[idx4plot,2],alpha = 0.5,bins = bins,stacked = True)
plt.legend(['ini','slip'])
ax2[5].set_title('max_lmr')



ax2[6].hist(ini_time[idx4plot,1] - ini_time[idx4plot,3] ,alpha = 0.5,bins = bins,stacked = True)
ax2[6].hist(slip_time[idx4plot,1] - slip_time[idx4plot,3],alpha = 0.5,bins = bins,stacked = True)
plt.legend(['ini','slip'])
ax2[6].set_title('min_lmr')



ax2[7].hist(ini_time[idx4plot,7]  ,alpha = 0.5,bins = bins,stacked = True)
ax2[7].hist(slip_time[idx4plot,7] ,alpha = 0.5,bins = bins,stacked = True)
plt.legend(['ini','slip'])
ax2[7].set_title('cm_y_dot')
# ax1.plot(ini_time[:,0],ini_time[:,1],'*')
# ax1.plot(slip_time[:,0],slip_time[:,1],'*')

plt.legend(['ini','slip'])


    # plt.plot(time[idx],grad[idx],'*')


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
idx4plot = np.where(np.logical_and(slip_time[:, 7] > 10, slip_time[:, 7] < -10))[0]
idx4plot

array([], dtype=int64)

In [ ]:


fig1,ax1 = plt.subplots(1,1)

ax1.plot(ini_time[idx4plot,7],(ini_time[idx4plot,0] - ini_time[idx4plot,1]) - (ini_time[idx4plot,2] - ini_time[idx4plot,3]) ,'*')
ax1.plot(slip_time[idx4plot,7],(slip_time[idx4plot,0] - slip_time[idx4plot,1]) - (slip_time[idx4plot,2] - slip_time[idx4plot,3]) ,'*')

plt.show()


fig3,ax3 = plt.subplots(1,1)

ax3.plot(ini_time[idx4plot,7],(ini_time[idx4plot,0] - ini_time[idx4plot,2] )  ,'*')
ax3.plot(slip_time[idx4plot,7],slip_time[idx4plot,1] - slip_time[idx4plot,3]  ,'*')

plt.show()

In [ ]:

fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('slip_angle_betaY','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)

fig.show()

KeyError: 'slip_angle_betaY'

In [ ]:
fig = go.Figure()

exp.plot_prop_movies('phi_rw_min_val','mean_body',fig,case = 'plot_mov',prop_x = 'time' ,
                     add_horizontal_line=None,t0 = 0,t1 = 60)

exp.plot_prop_movies('phi_lw_min_val','mean_body',fig,case = 'plot_mov',prop_x = 'time' ,
                     add_horizontal_line=None,t0 = 0,t1 = 60)
fig.show()

In [ ]:
exp = experiments['manipulated_2023_08_10_100ms']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']
fig = go.Figure()
exp.plot_prop_movies('freq_phi_rw_min_idx','mean_body',fig,case = 'plot_mov',prop_x = 'time' ,
                     add_horizontal_line=None)
fig.show()

In [ ]:

prop = 'q_dot'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]

[None, None, None, None, None, None]

In [ ]:
[exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_body','mean_mean_phi') for exp in experiments.values()] # calculate the mean value of two properties


[None, None, None, None, None, None]

In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']
phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)

fig = go.Figure()

exp.plot_prop_movies('phi_rw_min_val','mean_body',fig,case = 'plot_mov',prop_x = 'time' ,
                     add_horizontal_line=None)
fig.show()





fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_x_projected','mean_body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()
fig = go.Figure()
exp.plot_prop_movies('delta_ang_CM_dot_x','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)

fig.show()
fig = go.Figure()
exp.plot_prop_movies('freq_phi_rw_min_idx','mean_body',fig,case = 'plot_mov',prop_x = 'freq_phi_lw_min_idx' ,
                     add_horizontal_line=None,color = phi_r_color)
fig.show()

fig = go.Figure()
exp.plot_prop_movies('amp_rw','mean_body',fig,case = 'plot_mov',prop_x = 'phi_rw_min_val' ,
                     add_horizontal_line=None,mode = 'markers', color = phi_r_color)
exp.plot_prop_movies('amp_lw','mean_body',fig,case = 'plot_mov',prop_x = 'phi_lw_max_val' ,
                     add_horizontal_line=None,mode = 'markers',color = phi_l_color)
fig.show()

In [ ]:
exp = experiments['manipulated_2022_02_03_dark']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']

fig = go.Figure()

phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)
# exp.plot_prop_movies('amp_real2','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('delta_ang_CM_dot_x','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()




fig = go.Figure()

exp.plot_prop_movies('phi_rw_min_val','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,color = phi_r_color)
exp.plot_prop_movies('phi_lw_min_val','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,color = phi_l_color)

# exp.plot_prop_movies('phi_rw_max_val','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,color = phi_r_color)
# exp.plot_prop_movies('phi_lw_max_val','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,color = phi_l_color)
fig.show()


In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']

prop = 'delta_ang_Xax'
fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
exp.plot_prop_movies('delta_ang_CM_dot_x','body',fig,case = 'plot_mov',prop_x = 'time')

fig.show()



phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)

fig = go.Figure()
exp.plot_prop_movies('phi_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('phi_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)
fig.show()


Plot 3d trajectory

In [ ]:
mov = ['mov140','mov141','mov143','mov144','mov146','mov147']
exp = experiments['manipulated_2023_08_09_60ms']
exp.plot_3d_traj_movies('delta_ang_CM_dot_x',mov = mov)


In [ ]:


exp = experiments['manipulated_2023_08_09_60ms']
prop_name = 'delta_roll_yaw' #  np.abs( np.nanmean(prop[-300:])) >20:
dict_v = {'lt_zero':[],'gt_zero':[]}
for mov_nm in exp.mov_names:
    mov = exp.get_mov(mov_nm)
    prop = mov.get_prop(prop_name,'body')
    if np.abs( np.nanmean(prop[-300:])) >20:
        dict_v['lt_zero'].append(mov_nm )
    else:
        dict_v['gt_zero'].append(mov_nm )

fig = go.Figure()

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['lt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['gt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()


prop_name = 'CM_dot_y_projected_all_axes'
fig = go.Figure()

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['lt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['gt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()


prop_name = 'CM_dot_y_projected'
fig = go.Figure()

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['lt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['gt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()


prop_name = 'roll_body'
fig = go.Figure()

exp.plot_prop_movies(prop_name,'mean_body',fig,mov = dict_v['lt_zero'],case = 'plot_exp',prop_x = 'delta_amp',color = color_map[3],legend = f'cluster{1}',mode = 'markers')
exp.plot_prop_movies(prop_name,'mean_body',fig,mov = dict_v['gt_zero'],case = 'plot_exp',prop_x = 'delta_amp',color = color_map[4],legend = f'cluster{2}',mode = 'markers')
fig.show()



KeyError: 'delta_roll_yaw'

In [ ]:

prop = 'roll_body'
[exp.mean_by_stroke_movies(prop,'mean_body','body')  for exp in experiments.values()]


[None, None, None, None, None, None]

In [ ]:
prop_name = 'CM_dot_y_projected_all_axes'
fig = go.Figure()
exp = experiments['manipulated_2023_08_09_60ms']

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['lt_zero'],case = 'plot_mov',prop_x = 'time_m_zero_v',mode = 'markers')
# exp.plot_prop_movies(prop_name,'mean_body',fig,mov = dict_v['gt_zero'],case = 'plot_mov',prop_x = 'delta_amp',color = color_map[4],mode = 'markers')
fig.show()

In [ ]:
prop_name = 'delta_ang_Xax'
fig = go.Figure()

exp.plot_prop_movies(prop_name,'mean_body',fig,mov = dict_v['lt_zero'],case = 'plot_exp',prop_x = 'delta_amp',color = color_map[3],legend = f'cluster{1}',mode = 'markers')
exp.plot_prop_movies(prop_name,'mean_body',fig,mov = dict_v['gt_zero'],case = 'plot_exp',prop_x = 'delta_amp',color = color_map[4],legend = f'cluster{2}',mode = 'markers')
fig.show()

In [ ]:
dict_v['lt_zero']

['mov101',
 'mov128',
 'mov133',
 'mov136',
 'mov137',
 'mov143',
 'mov153',
 'mov165',
 'mov22',
 'mov26',
 'mov3',
 'mov36',
 'mov51',
 'mov56',
 'mov61',
 'mov62',
 'mov64',
 'mov71',
 'mov78',
 'mov79',
 'mov80',
 'mov91',
 'mov94']

In [ ]:
mov = exp.get_mov('mov8')
ay = mov.get_prop('CM_dot_dot_y_projected_all_axes','body')
ax = mov.get_prop('CM_dot_dot_x_projected_all_axes','body')
az = mov.get_prop('CM_real_z_body_dot_dot_smth','body')


cm = mov.get_prop('CM_real_x_body','body',three_col=2)

force = np.hstack((ax,ay))
force = (force.T/np.linalg.norm(force,axis = 1)).T

exp = experiments['manipulated_2023_08_09_60ms']
mov = ['mov60']
exp.plot_3d_traj_movies('CM_dot_x_projected',mov = mov,force = force)


# skp = 50
# plt.scatter(cm[:,0],cm[:,1])
# plt.quiver(cm[::skp,0],cm[::skp,1],force[::skp,0],force[::skp,1])



In [ ]:

def rotation_matrix(yaw,pitch,roll):
    roll_mat = np.vstack([[1,0,0],[0 ,np.cos(roll),-np.sin(roll)],[0, np.sin(roll), np.cos(roll)]])
    pitch_mat = np.vstack([[np.cos(pitch),0,np.sin(pitch)],[0, 1,0],[-np.sin(pitch), 0, np.cos(pitch)]])
    yaw_mat = np.vstack([[np.cos(yaw),-np.sin(yaw),0],[np.sin(yaw),np.cos(yaw),0],[0, 0, 1]])
    return yaw_mat @ pitch_mat @ roll_mat


for exp_name in experiments.keys():
    exp = experiments[exp_name]
    wing_body = 'body'
    for mov_name in exp.mov_names:
        mov = exp.get_mov(mov_name)

        yaw = mov.get_prop('yaw_body','body')[:,0]
        pitch = mov.get_prop('pitch_body','body')[:,0]
        roll = mov.get_prop('roll_body','body')[:,0]
        time = mov.get_prop('time','body')[:,0]

        rot_mat = [rotation_matrix(yaw*np.pi/180,-pitch*np.pi/180,roll*np.pi/180).T for yaw,pitch,roll in zip(yaw,pitch,roll)]
        vxyz = mov.get_prop('CM_real_x_body_dot_smth','body',three_col=3)
        axyz = mov.get_prop('CM_real_x_body_dot_dot_smth','body',three_col=3)

        uvw = np.vstack([rot_mat @ vxyz for rot_mat,vxyz in zip(rot_mat,vxyz)])
        uvw_dot = np.vstack([rot_mat @ axyz for rot_mat,axyz in zip(rot_mat,axyz)])

        vz_sp = uvw[:,0]*np.cos(np.radians(45)) + uvw[:,2]*np.cos(np.radians(45))
        vx_sp = uvw[:,0]*np.cos(np.radians(45)) - uvw[:,2]*np.cos(np.radians(45))



        az_sp = uvw_dot[:,0]*np.cos(np.radians(45)) + uvw_dot[:,2]*np.cos(np.radians(45))
        ax_sp = uvw_dot[:,0]*np.cos(np.radians(45)) - uvw_dot[:,2]*np.cos(np.radians(45))

        mov.data[wing_body] = np.hstack((mov.data[wing_body], uvw))
        mov.add_to_header(['u','v','w'],wing_body)

        
        mov.data[wing_body] = np.hstack((mov.data[wing_body], uvw_dot))
        mov.add_to_header(['u_dot','v_dot','w_dot'],wing_body)

        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, vz_sp)).T
        mov.add_to_header(['vz_sp'],wing_body)
        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, vx_sp)).T
        mov.add_to_header(['vx_sp'],wing_body)

        
        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, az_sp)).T
        mov.add_to_header(['az_sp'],wing_body)
        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, ax_sp)).T
        mov.add_to_header(['ax_sp'],wing_body)
# fig,ax = plt.subplots(2,1)
# ax[0].plot(time,vx_sp)
# ax[0].plot(time,uvw[:,1])
# ax[0].plot(time,vz_sp)

# ax[1].plot(vxyz)
    
# for mov_name in exp.mov_names:

    
    # mov.data[wing_body] = np.hstack((mov.data[wing_body], uvw))
    # mov.add_to_header(['u','v','w'],wing_body)

    # mov.data[wing_body] = np.vstack((mov.data[wing_body].T, asp_z)).T
    # mov.add_to_header(['asp_z'],wing_body)


KeyboardInterrupt: 

In [ ]:
for name in ['ax_sp','az_sp','v_dot']:
    f,tm = exp.mean_time_series_prop(name,wing_body,window = (73*7)//2,dt = 1/16000)
    plt.plot(tm,f)

g:\Documents\experiment_analysis\data_analysis\Experiment.py:136: RuntimeWarning:

Mean of empty slice



Plot property and add interest point

In [ ]:

exp = experiments['manipulated_2023_08_06_40ms']
prop = 'CM_dot_dot_x_projected'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_dot_x_projected_peaks_max',prop,fig) 
fig.show()



In [ ]:
fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('CM_dot_dot_y_projected','body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)

fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('phi_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('phi_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)

fig.show()

3D trajectory plot

In [ ]:

exp = experiments['manipulated_2023_08_09_60ms']
mov = ['mov60']
exp.plot_3d_traj_movies('CM_dot_x_projected',mov = mov)


Plot histograms of peaks, plot movies time seires data with the max/min/peak data point on them

In [ ]:
# Histograms
point_name = 'pitch_y_frame_dot_dot_peaks_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()

point_name = 'pitch_y_frame_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'amp_v_peaks_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_projected_zero_v'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()

KeyError: 'CM_dot_dot_projected_min'

In [ ]:
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('CM_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('amp_v_peaks_min','amp_v',fig) 
fig.show()

In [ ]:
# time series of movies with min/max on them
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','pitch_y_frame_dot_dot',fig) 
fig.show()

fig = go.Figure()
exp.plot_prop_movies('CM_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','CM_dot_projected',fig) 
fig.show()

In [ ]:

fig = go.Figure()
exp.plot_prop_movies('roll_x_frame','body',fig,case = 'plot_mov',prop_x = 'time')
# exp.add_point_to_plot_movies('CM_dot_projected_zero_v','CM_dot_y_projected',fig) 
exp.plot_prop_movies('yaw_z_frame','body',fig,case = 'plot_mov',prop_x = 'time',yaxis = 'y2')
# exp.add_point_to_plot_movies('CM_dot_projected_zero_v','roll_body',fig,yaxis = 'y2') 

# yax_dict[f"yaxis2"] = (dict(title=prop,titlefont=dict(color=color),tickfont=dict(color=color),side = side[i],position=position[i]))



fig.update_layout(title = exp.experiment_name,
    xaxis=dict(domain=[0, 0.95]),
    yaxis=dict(title='roll_x_frame', titlefont=dict(color='red'), tickfont=dict(color='red'), side='left', position=0),
    yaxis2=dict(title='yaw_y_frame', titlefont=dict(color='green'), tickfont=dict(color='green'), side='right', position=0.95),
)

fig.show()

Plot all experiments - time series

In [ ]:
fig = go.Figure()
prop = 'pitch_y_frame_dot_dot'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()

Calculate and plot mean property (time series,  mean per frame of all movies)

In [ ]:
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'r'
prop = 'CM_dot_dot_projected'
props = ['delta_ang','delta_ang']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'vectors')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean, label = exp.experiment_name)


plt.legend()

g:\Documents\experiment_analysis\data_analysis\Experiment.py:121: RuntimeWarning:

Mean of empty slice



In [ ]:
mov.header['mean_body']

{'pitch_body': 0,
 'yaw_body': 1,
 'roll_body': 2,
 'pitch_body_dot': 3,
 'yaw_body_dot': 4,
 'roll_body_dot': 5,
 'pitch_body_dot_dot': 6,
 'yaw_body_dot_dot': 7,
 'roll_body_dot_dot': 8,
 'frames': 9,
 'time': 10,
 'CM_real_x_body': 11,
 'CM_real_y_body': 12,
 'CM_real_z_body': 13,
 'phi_rw_max_idx': 14,
 'phi_rw_max_val': 15,
 'phi_lw_max_idx': 16,
 'phi_lw_max_val': 17,
 'phi_rw_min_idx': 18,
 'phi_rw_min_val': 19,
 'phi_lw_min_idx': 20,
 'phi_lw_min_val': 21,
 'yaw_z_frame': 22,
 'pitch_y_frame': 23,
 'roll_x_frame': 24,
 'mean_idx': 25,
 'freq_phi_rw_min_idx': 26,
 'CM_dot_dot_x_projected': 27,
 'delta_ang_CM_dot_x': 28,
 'delta_ang_Xax': 29}

Try PCA

In [ ]:
exp.mov_names

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.cluster import KMeans

mov_feat = []
props = ['CM_dot_dot_x_projected','freq_phi_rw_min_idx']

exp_pd = []
for exp in list(experiments.values())[1:2]:
    mov_feat = []
    for idx,mov_name in enumerate(exp.mov_names):
        mov = exp.get_mov(mov_name)
        mov_feat.append(mov.data['mean_body'])

    featpd_exp = pd.DataFrame(data = np.vstack(mov_feat),columns = mov.header['mean_body'])
    featpd_exp['exp_name'] = exp.experiment_name
    # featpd_exp['mov'] = exp.mov_names
    exp_pd.append(featpd_exp)

feat = pd.concat(exp_pd,axis = 0).dropna().reset_index()


dataforpca = feat[props]
n_comp = len(props)
pca = PCA(n_components=n_comp)
principalComponents = pca.fit_transform(dataforpca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = [f'pca_{idx}' for idx in range(n_comp)])

pca_name = [f'pca_{idx}' for idx in range(pca.n_components_)]


featpd = pd.concat([feat, principalDf], axis = 1)
kmeans = KMeans(n_clusters=n_comp, random_state=0, n_init="auto").fit(featpd[pca_name])
print(kmeans.labels_)
pca.components_


[0 0 0 ... 0 0 0]


array([[-9.89891583e-04,  9.99999510e-01],
       [-9.99999510e-01, -9.89891583e-04]])

In [ ]:
for lbl in np.unique(kmeans.labels_):
    feat.iloc[kmeans.labels_==lbl]['delta_ang_CM_dot_x'].plot(marker = '*')

In [ ]:

fig = go.Figure()

for lbl in np.unique(kmeans.labels_):
    exp_names = np.unique(feat.iloc[kmeans.labels_==lbl]['exp_name'])
    feat_lbl = feat.iloc[kmeans.labels_==lbl]
    for idx,exp_name in enumerate(exp_names): 
        mov_exp_lbl = list(feat_lbl.loc[feat_lbl['exp_name'] == exp_name])

        prop = 'pitch_y_frame_dot'
        prop = 'CM_dot_dot_projected'
        # prop = 'pitch_body'

        color_map = colormap.datad["tab10"]['listed']

        experiments[exp_name].plot_prop_movies(prop,'body',fig,mov = mov_exp_lbl,case = 'plot_exp',prop_x = 'time',color = color_map[lbl],legend = f'{exp_name}_cluster{lbl}')
fig.show()
# feat.iloc[kmeans.labels_==lbl]['exp_name'] == exp_names[0]
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'q'
prop = 'CM_dot_dot_projected'
props = ['pitch_y_frame_dot_dot','pitch_y_frame_dot']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'body')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean)

KeyError: 'index'

Plot and save figures

In [ ]:
        plot_and_save_movie_prop(exp, 'CM_dot_dot_x_projected','body','acceleration projected on XY',add_horizontal_line = 0)


In [ ]:
import plotly.graph_objects as go
import plotly
import matplotlib.cm as colormap
color_map = colormap.datad["tab10"]['listed']



def plot_and_save_movie_prop(exp,yprop,wing_body,title,add_horizontal_line = None):
    
    fig = go.Figure()

    exp.plot_prop_movies(yprop,wing_body,fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = add_horizontal_line)
    fig.update_layout(title = title)
   
    fig.show()
    plotly.offline.plot(fig, filename=f'{exp.figures_path}/{yprop}_plot.html',auto_open=False)




for exp in experiments:
    try:
        exp = experiments[exp]
        exp.plot_3d_traj_movies('CM_dot_x_projected',save_plot = True, mov = False)

        plot_and_save_movie_prop(exp, 'slip_angle_beta', 'vectors','slip angle [ang]', add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'delta_ang_CM_dot_x','vectors','change in velocity direction [ang]',add_horizontal_line = None)
        plot_and_save_movie_prop(exp, 'delta_ang_Xax','vectors','change in body axis [ang]',add_horizontal_line = None)

        plot_and_save_movie_prop(exp, 'CM_dot_dot_x_projected','body','acceleration projected on XY',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'pitch_body','body','pitch angle', add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'pitch_body_dot','body','pitch_dot', add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'CM_dot_x_projected','body','velocity projected on XY', add_horizontal_line = 0)
    except:
        continue
    # exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi')
    # exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi')

    # plot_and_save_movie_prop(exp,yprop = 'mean_front_phi',wing_body = 'mean_wing')
    # plot_and_save_movie_prop(exp,yprop = 'mean_mean_phi',wing_body = 'mean_wing')



In [ ]:
def add_axis(axis,fig,line_width,sp,color):
    origin = [0, 0, 0]
    x_end = origin + axis
    # Create traces for each axis
    fig.add_trace(go.Scatter3d(x=[origin[0], x_end[0]], y=[origin[1], x_end[1]], z=[origin[2], x_end[2]], 
                               mode='lines', name=f'{sp}',line=dict(width=line_width,color = color)))
    return fig

def plot_rotation_matrix_axes(rotation_matrix,fig,sp,line_width):
    # Extract axes vectors
    x_axis = rotation_matrix[:, 0]
    y_axis = rotation_matrix[:, 1]
    z_axis = rotation_matrix[:, 2]
    
    # Define endpoints of axes
  
    # Create traces for each axis
    fig = add_axis(x_axis,fig,line_width,f'X-axis_{sp}','red')
    fig = add_axis(y_axis,fig,line_width,f'Y-axis_{sp}','green')
    fig = add_axis(z_axis,fig,line_width,f'Z-axis_{sp}','blue')
    
    return fig
    # Create figure and plot
   

frame = 2000 - 274

fig = go.Figure()
fig = plot_rotation_matrix_axes(np.dot(rot_mat[frame],rot_mat_sp)*10,fig,'sp',line_width = 3)
fig = plot_rotation_matrix_axes(rot_mat[frame]*10,fig,'body',line_width = 10)
fig = add_axis(acc[frame],fig,5,'acc','pink')
fig = add_axis((acc_calc[frame]),fig,5,'acc_z_sp','black')
fig = add_axis((acc_calc[frame] - np.array([0,0,1])*9.8),fig,5,'acc_z_sp - g_zlab','black')

fig.show()

# Plot the axes
